In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
openc

In [ ]:
%pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.1 MB/s eta 0:00:00
^C


In [ ]:
%pip install "numpy<2"
%pip install spacy
spacy.cli.download("en_core_web_sm")

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 29 13:54:48 2025

@author: dnd2129
NLP processes
"""

import pandas as pd

import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import punkt
from nltk import word_tokenize, ngrams
from nltk.util import ngrams

import spacy

import gensim
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools

#import pyLDAvis
#import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import auth
import gspread
from google.auth import default

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/raw data/Project - Student Course Clustering/Student Course Elective Enrollments Graduates 2016-2025.xlsx")
df.head()
#unq_courses= df.combined_name.unique()



,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined Native Flag,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,International,International,International,International,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities


#NLP Process

## Cleaning Text

In [ ]:

#cleaning texts
df['clean_name'] = df['combined_name'].fillna('') #changing nan to blank
df['clean_name'] = df['clean_name'].str.replace(r'\s+', ' ', regex=True)#cleaning new line chars
df['clean_name'] = df['clean_name'].str.replace("'", '', regex=False)#removing single quotes

df.head()


,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,International,International,International,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities


### Remove stopwords

In [ ]:
def rem_sw(var_in):
    sw = list(set(stopwords.words('english')))
    tmp = [word for word in var_in.split() if word not in sw]
    return ' '.join(tmp)

In [ ]:
df['clean_name'] = df['clean_name'].apply(rem_sw)
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,White,White,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,International,International,International,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities


In [ ]:
df.tail()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
147043,147043,C004444318,ep2893,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,H-,...,International,International,International,Netherlands,NaN,Netherlands,Consulting,Consulting,NaN,
147044,147044,C004444374,st3389,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,H-,...,International,International,International,Italy,NaN,Italy,Consulting,Media / Communications,NaN,
147045,147045,C004455657,tpl2116,2022,20213MGMTB8584001,NaN,MGMTB8584,MANAGEMENT,1.5,HP-,...,International,International,International,United Kingdom,NaN,United Kingdom,Entrepreneur/Owner,Real Estate,NaN,
147046,147046,C004443909,au2257,2023,20221DROMB8103061,NaN,DROMB8103,"DEC,RISK & OPERATIONS MGT",1.5,A-,...,International,International,International,Brazil,Mexico,Brazil,Consulting,Consulting,NaN,
147047,147047,C004536545,sdv2118,2023,20221DROMB8103061,NaN,DROMB8103,"DEC,RISK & OPERATIONS MGT",1.5,B+,...,Not Native Hawaiian or Other Pacific Islander,Not White,Asian,United States,NaN,United States,Consulting,Consulting,NaN,


## n-gram extraction

### Manual Tokenizing::: may be ignore as tfidf may be better!

In [ ]:
'''
Tokenize each sentence into a list of words, removing punctuations and
unnecessary characters altogether.

Using spaCy because nltk wasn't working-- and according to article, spaCy is better anyway....
'''
nlp = spacy.load("en_core_web_sm")

def generate_ngrams_spacy(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_punct and not token.is_space]

    unigrams = list(zip(tokens))
    bigrams = list(zip(tokens, tokens[1:])) if len(tokens) > 1 else []
    trigrams = list(zip(tokens, tokens[1:], tokens[2:])) if len(tokens) > 2 else []

    return unigrams, bigrams, trigrams


In [ ]:
df[['unigrams', 'bigrams', 'trigrams']] = df['clean_name'].fillna('').apply(
    lambda x: pd.Series(generate_ngrams_spacy(x))
)

KeyboardInterrupt: 

In [ ]:
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name,token_name,unigrams,bigrams,trigrams
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1315...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1315...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1502...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1502...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities,<generator object sent_to_words at 0x7d6ca1503...,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"


In [ ]:
#df.to_csv('course names uni-bi-trigrams.csv')

In [ ]:
subset = df[['clean_name', 'unigrams', 'bigrams', 'trigrams']].copy()

# Convert unhashable list columns to strings
subset['unigrams'] = subset['unigrams'].astype(str)
subset['bigrams'] = subset['bigrams'].astype(str)
subset['trigrams'] = subset['trigrams'].astype(str)

# Now drop duplicates safely
unique_subset = subset.drop_duplicates()

# Check it
unique_subset.head()


,clean_name,unigrams,bigrams,trigrams
0,Winning Strategic Capabilities,"[('winning',), ('strategic',), ('capabilities',)]","[('winning', 'strategic'), ('strategic', 'capa...","[('winning', 'strategic', 'capabilities')]"
266,Web App Programming Python,"[('web',), ('app',), ('programming',), ('pytho...","[('web', 'app'), ('app', 'programming'), ('pro...","[('web', 'app', 'programming'), ('app', 'progr..."
581,Virtual Reality & The Future Storytelling: How...,"[('virtual',), ('reality',), ('the',), ('futur...","[('virtual', 'reality'), ('reality', 'the'), (...","[('virtual', 'reality', 'the'), ('reality', 't..."
742,Virtual Reality & Artificial Intelligence: How...,"[('virtual',), ('reality',), ('artificial',), ...","[('virtual', 'reality'), ('reality', 'artifici...","[('virtual', 'reality', 'artificial'), ('reali..."
817,Virtual Reality & Artificial Intelligence,"[('virtual',), ('reality',), ('artificial',), ...","[('virtual', 'reality'), ('reality', 'artifici...","[('virtual', 'reality', 'artificial'), ('reali..."


In [ ]:
#unique_subset.to_csv("unique_subset.csv", index=False)

Manually looking at unigrams, bigrams and trigrams to decide which to use for LDA

### Tf-idf

In [ ]:
# Your existing setup
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(df['clean_name'])

# Sum TF-IDF values across all documents
tfidf_sums = X.sum(axis=0).A1  # A1 flattens the matrix to a 1D array
feature_names = vectorizer.get_feature_names_out()

gives feature names, i.e., ngrams

getting  SCORE FOR EACH ngram

In [ ]:
# Create DataFrame of words and scores
tfidf_scores = pd.DataFrame({
    'term': feature_names,
    'score': tfidf_sums
})

# Sort by score descending
tfidf_scores = tfidf_scores.sort_values(by='score', ascending=False)

# View top words/phrases
print(tfidf_scores.head(30))

                             term        score
1870                   managerial  3291.995459
1874      managerial negotiations  3210.213805
2093                 negotiations  3210.213805
1097                      finance  3071.540406
2698                     strategy  3069.928453
1946                      markets  3036.397317
124                      analysis  2875.461116
1621                    investing  2742.828802
418                       capital  2625.593284
1667                  investments  2595.576559
424               capital markets  2589.281199
1953          markets investments  2580.466807
425   capital markets investments  2580.466807
468                        change  2152.975640
2790                          the  2074.443045
350                      business  2068.745496
1843                   management  2016.757233
2160               organizational  1961.051792
1163                  foundations  1958.536477
2161        organizational change  1953.261068
2980         

write csv

In [ ]:
tfidf_scores.to_csv("tfidf_scores ngrams 1,2,3.csv", index=False)

### IDF: maybe this will make it easier to sort out words!
higher scores--> less frequent-- may be more valuable to us

In [ ]:
import math
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Use CountVectorizer to get n-grams (1 to 3)
vectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='word')
X = vectorizer.fit_transform(df['clean_name'])

# Step 2: Extract n-gram feature names
feature_names = vectorizer.get_feature_names_out()

# Step 3: Get number of documents each n-gram appears in (document frequency)-- I think this is counting each ngram list as a document, and NOT the clean course name.
# This is a binary presence matrix — whether term appears in document
document_frequencies = (X > 0).sum(axis=0).A1  # .A1 flattens to 1D array
num_documents = X.shape[0]


In [ ]:
print(feature_names)

['101' '101 driving' '101 driving strategic' ... 'your startup'
 'your startup ii' 'your startup tech']


In [ ]:
# Step 4: Compute smoothed IDF scores
idf_scores = {
    term: math.log((1 + num_documents) / (1 + dfreq)) + 1
    for term, dfreq in zip(feature_names, document_frequencies)
}

In [ ]:
# Step 5: Convert to DataFrame and sort
idf_df = pd.DataFrame(idf_scores.items(), columns=["term", "idf_score"])
idf_df['total_count_across_ngrams'] = document_frequencies #adding number of documents(coursename) each term appears in
#matches substring and \b ..\b matches word boundaries... so 'database' won't count in 'data'.
#This is re-countign even if it occurs in shorter/bigger ngrams for the same name--- need to get just count.
idf_df = idf_df.sort_values(by="idf_score", ascending=False)

idf_df.head(20)



,term,idf_score,df_count
0,101,12.205374,1
1,101 driving,12.205374,1
1370,high tech firms,12.205374,1
1418,immersion exploring turkeys,12.205374,1
1495,in financial,12.205374,1
1496,in financial markets,12.205374,1
1517,industries integrating,12.205374,1
1518,industries integrating mba,12.205374,1
1522,inference,12.205374,1
1529,information in,12.205374,1


#### Add count of unique course_identifiers which have each term

 Re-doing clean name with ngrams list linked to clean names using ccount vetorizer instead of manual spacy command like under 'manual tokenization'

In [ ]:
def generate_ngrams_countvectorizer(text):
  # Combine into a list since CountVectorizer expects an iterable of docs
  docs = [text.lower()]

  # Unigrams
  vectorizer_uni = CountVectorizer(ngram_range=(1, 1))
  unigrams = vectorizer_uni.fit(docs)
  unigrams_list = list(unigrams.vocabulary_.keys())

  # Bigrams
  vectorizer_bi = CountVectorizer(ngram_range=(2, 2))
  bigrams = vectorizer_bi.fit(docs)
  bigrams_list = list(bigrams.vocabulary_.keys())

  # Trigrams
  vectorizer_tri = CountVectorizer(ngram_range=(3, 3))
  trigrams = vectorizer_tri.fit(docs)
  trigrams_list = list(trigrams.vocabulary_.keys())

  return unigrams_list, bigrams_list, trigrams_list
# use
df[['unigrams', 'bigrams', 'trigrams']] = df['clean_name'].fillna('').apply(
    lambda x: pd.Series(generate_ngrams_spacy(x))
)

In [ ]:
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name,unigrams,bigrams,trigrams
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,United States,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,United States,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,United States,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,United States,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,Singapore,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]"


In [ ]:
subset = df[['clean_name', 'unigrams', 'bigrams', 'trigrams']].copy()

# Convert unhashable list columns to strings
subset['unigrams'] = subset['unigrams'].astype(str)
subset['bigrams'] = subset['bigrams'].astype(str)
subset['trigrams'] = subset['trigrams'].astype(str)

# Now drop duplicates safely
unique_subset = subset.drop_duplicates()

# Check it
unique_subset.head()


,clean_name,unigrams,bigrams,trigrams
0,Winning Strategic Capabilities,"[('winning',), ('strategic',), ('capabilities',)]","[('winning', 'strategic'), ('strategic', 'capa...","[('winning', 'strategic', 'capabilities')]"
266,Web App Programming Python,"[('web',), ('app',), ('programming',), ('pytho...","[('web', 'app'), ('app', 'programming'), ('pro...","[('web', 'app', 'programming'), ('app', 'progr..."
581,Virtual Reality & The Future Storytelling: How...,"[('virtual',), ('reality',), ('the',), ('futur...","[('virtual', 'reality'), ('reality', 'the'), (...","[('virtual', 'reality', 'the'), ('reality', 't..."
742,Virtual Reality & Artificial Intelligence: How...,"[('virtual',), ('reality',), ('artificial',), ...","[('virtual', 'reality'), ('reality', 'artifici...","[('virtual', 'reality', 'artificial'), ('reali..."
817,Virtual Reality & Artificial Intelligence,"[('virtual',), ('reality',), ('artificial',), ...","[('virtual', 'reality'), ('reality', 'artifici...","[('virtual', 'reality', 'artificial'), ('reali..."


In [ ]:
import ast

unique_subset['unigrams'] = unique_subset['unigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
unique_subset['bigrams'] = unique_subset['bigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
unique_subset['trigrams'] = unique_subset['trigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


/tmp/ipython-input-1842502005.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_subset['unigrams'] = unique_subset['unigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
/tmp/ipython-input-1842502005.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_subset['bigrams'] = unique_subset['bigrams'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
/tmp/ipython-input-1842502005.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [ ]:
def flatten_and_join(row):
    def tuple_to_str(t):
        if isinstance(t, (tuple, list)):
            return " ".join(t)
        elif isinstance(t, str):
            return t
        else:
            return ""

    def safe_get(col):
        val = row[col]
        if isinstance(val, list):
            return val
        else:
            return []

    unigrams = safe_get('unigrams')
    bigrams = safe_get('bigrams')
    trigrams = safe_get('trigrams')

    all_ngrams = [tuple_to_str(t) for t in (unigrams + bigrams + trigrams) if t]
    return ", ".join(all_ngrams)

unique_subset['all_ngrams'] = unique_subset.apply(flatten_and_join, axis=1)

unique_subset.head()

/tmp/ipython-input-175960449.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_subset['all_ngrams'] = unique_subset.apply(flatten_and_join, axis=1)


,clean_name,unigrams,bigrams,trigrams,all_ngrams,clean_ngrams
0,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]","winning, strategic, capabilities, winning stra...","[w, i, n, n, i, n, g, ,, , s, t, r, a, t, e, ..."
266,Web App Programming Python,"[(web,), (app,), (programming,), (python,)]","[(web, app), (app, programming), (programming,...","[(web, app, programming), (app, programming, p...","web, app, programming, python, web app, app pr...","[w, e, b, ,, , a, p, p, ,, , p, r, o, g, r, ..."
581,Virtual Reality & The Future Storytelling: How...,"[(virtual,), (reality,), (the,), (future,), (s...","[(virtual, reality), (reality, the), (the, fut...","[(virtual, reality, the), (reality, the, futur...","virtual, reality, the, future, storytelling, h...","[v, i, r, t, u, a, l, ,, , r, e, a, l, i, t, ..."
742,Virtual Reality & Artificial Intelligence: How...,"[(virtual,), (reality,), (artificial,), (intel...","[(virtual, reality), (reality, artificial), (a...","[(virtual, reality, artificial), (reality, art...","virtual, reality, artificial, intelligence, ho...","[v, i, r, t, u, a, l, ,, , r, e, a, l, i, t, ..."
817,Virtual Reality & Artificial Intelligence,"[(virtual,), (reality,), (artificial,), (intel...","[(virtual, reality), (reality, artificial), (a...","[(virtual, reality, artificial), (reality, art...","virtual, reality, artificial, intelligence, vi...","[v, i, r, t, u, a, l, ,, , r, e, a, l, i, t, ..."


In [ ]:
# Step 3: Split combined n-grams into individual terms
unique_subset['all_ngrams'] = unique_subset['all_ngrams'].apply(lambda x: [t.strip() for t in x.split(',') if t.strip()])
unique_subset.head()

/tmp/ipython-input-3474425674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_subset['all_ngrams'] = unique_subset['all_ngrams'].apply(lambda x: [t.strip() for t in x.split(',') if t.strip()])


,clean_name,unigrams,bigrams,trigrams,all_ngrams,clean_ngrams
0,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]","[winning, strategic, capabilities, winning str...","[w, i, n, n, i, n, g, ,, , s, t, r, a, t, e, ..."
266,Web App Programming Python,"[(web,), (app,), (programming,), (python,)]","[(web, app), (app, programming), (programming,...","[(web, app, programming), (app, programming, p...","[web, app, programming, python, web app, app p...","[w, e, b, ,, , a, p, p, ,, , p, r, o, g, r, ..."
581,Virtual Reality & The Future Storytelling: How...,"[(virtual,), (reality,), (the,), (future,), (s...","[(virtual, reality), (reality, the), (the, fut...","[(virtual, reality, the), (reality, the, futur...","[virtual, reality, the, future, storytelling, ...","[v, i, r, t, u, a, l, ,, , r, e, a, l, i, t, ..."
742,Virtual Reality & Artificial Intelligence: How...,"[(virtual,), (reality,), (artificial,), (intel...","[(virtual, reality), (reality, artificial), (a...","[(virtual, reality, artificial), (reality, art...","[virtual, reality, artificial, intelligence, h...","[v, i, r, t, u, a, l, ,, , r, e, a, l, i, t, ..."
817,Virtual Reality & Artificial Intelligence,"[(virtual,), (reality,), (artificial,), (intel...","[(virtual, reality), (reality, artificial), (a...","[(virtual, reality, artificial), (reality, art...","[virtual, reality, artificial, intelligence, v...","[v, i, r, t, u, a, l, ,, , r, e, a, l, i, t, ..."


In [ ]:
#Just for quick future use.
#unique_subset.to_csv('/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/CV_unique uni-bi-trigram_subset.csv')

In [ ]:

# Step 4: Flatten into a single list of terms
all_terms = [term for terms_list in unique_subset['all_ngrams'] for term in terms_list]

# Step 5: Count term occurrences
term_counts = pd.Series(all_terms).value_counts().reset_index()
term_counts.columns = ['term', 'count']
term_counts.head()

,term,count
0,immersion,63
1,global,57
2,business,52
3,the,52
4,management,40


In [ ]:
term_counts.tail()

,term,count
3116,seminar creating,1
3117,creating value,1
3118,value healthcare,1
3119,healthcare delivery,1
3120,applied multivariate statistics,1


In [ ]:
# Step 6: Merge counts back into your idf_df
idf_with_counts = idf_df.merge(term_counts, on='term', how='left')
idf_with_counts.head()

,term,idf_score,df_count,count
0,101,12.205374,1,1.0
1,101 driving,12.205374,1,1.0
2,high tech firms,12.205374,1,1.0
3,immersion exploring turkeys,12.205374,1,1.0
4,in financial,12.205374,1,1.0


In [ ]:
idf_with_counts.tail(10)

,term,idf_score,df_count,count
3063,investments,4.151487,6291,3.0
3064,capital,4.079743,6759,9.0
3065,management,3.993348,7369,40.0
3066,finance,3.770476,9209,23.0
3067,investing,3.765818,9252,38.0
3068,business,3.753573,9366,52.0
3069,analysis,3.718640,9699,13.0
3070,strategy,3.696616,9915,37.0
3071,markets,3.652331,10364,30.0
3072,the,3.538468,11614,52.0


In [ ]:
# Step 6: Export to CSV
idf_with_counts.to_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/idf_ngram_COUNT.csv", index=False)



#### What if i remove verbs?

In [ ]:
from nltk.tag import pos_tag

import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
tagged_tokens = pos_tag(idf_with_counts['term'])

In [ ]:
idf_with_counts["non_verb_tokens"] = [
    word if not tag.startswith('VB') else ''  # or None if you prefer
    for word, tag in tagged_tokens
]
idf_with_counts.head()

,term,idf_score,df_count,count,noVB,non_verb_tokens,tagged_tokens
0,101,12.205374,1,1.0,101,101,"(101, CD)"
1,101 driving,12.205374,1,1.0,101 driving,101 driving,"(101 driving, CD)"
2,high tech firms,12.205374,1,1.0,high tech firms,high tech firms,"(high tech firms, NNS)"
3,immersion exploring turkeys,12.205374,1,1.0,in financial,,"(immersion exploring turkeys, VBP)"
4,in financial,12.205374,1,1.0,in financial markets,in financial,"(in financial, JJ)"


### RAKE/YAKE/ Keyword extraction

#### RAKE (Rapid Automatic Keyword extraction)

In [ ]:
%pip install rake-nltk


In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from rake_nltk import Rake

rake = Rake()

def extract_rake_keywords(text):
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()

df["rake_keywords"] = df["clean_name"].apply(extract_rake_keywords)
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name,unigrams,bigrams,trigrams,rake_keywords
0,0,C003535762,ps2618,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP,...,NaN,United States,Management Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]",[winning strategic capabilities]
1,1,C003536543,mag2343,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,H-,...,NaN,United States,Consulting,Consulting,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]",[winning strategic capabilities]
2,2,C003547460,wpg2107,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,NaN,United States,Entrepreneur/Owner,Entertainment/Leisure/Sports,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]",[winning strategic capabilities]
3,3,C003563297,mjm2364,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP+,...,NaN,United States,Fin. - Investment Portfolio Management,Fin Serv: Investment/Portfolio Management,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]",[winning strategic capabilities]
4,4,C003577787,kh2929,2020,20193MRKTB8625001,Elective,MRKTB8625,MARKETING,3.0,HP-,...,NaN,Singapore,Fin. - Venture Capital/Private Equity,Fin Serv: Private Equity / Venture Capital,Winning Strategic Capabilities,Winning Strategic Capabilities,"[(winning,), (strategic,), (capabilities,)]","[(winning, strategic), (strategic, capabilities)]","[(winning, strategic, capabilities)]",[winning strategic capabilities]


#### YAKE (Yet Another Keyword Extractor)

In [ ]:
%pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 8.0 MB/s eta 0:00:00


In [ ]:
import yake
#YAKE gives all words in ranked order with scores to highlight important keywords-- I just want the top 5 keywords (for simplicity) and i dont care about the score.

kw_extractor = yake.KeywordExtractor(top=5)  # limit to top 5 keywords

def extract_top5_yake_keywords(text):
    keywords = kw_extractor.extract_keywords(text)  # list of (keyword, score) tuples
    # Extract only the keywords (ignore scores)
    return [kw for kw, score in keywords]

df["yake_top5_keywords"] = df["clean_name"].apply(extract_top5_yake_keywords)

In [ ]:
df["yake_top5_keywords"].iloc[0]

['Winning Strategic Capabilities',
 'Strategic Capabilities',
 'Winning Strategic',
 'Capabilities',
 'Strategic']

In [ ]:
df.to_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/ngrams_rake_yake_keywords.csv")

# LDA processes

now, combine all courses for each student into one row so we have one row per studnt. Then,tokenize it like I did above (choose uni/bi or trigrams depending of which worked best).
THEN: only retain the tokens that I manually chose to use from the manual mapping.

### Grouping by student
One row per student-- adding a course_list column which contains all courses they have taken.

In [ ]:
'''
Check the Sparsicity
'''
# Materialize the sparse data
data_dense = data_vectorized.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")
#Sparsicity:  0.6425770811326413 %

Sparsicity:  0.6425770811326413 %


In [ ]:
#One row per student-- adding a course_list column which contains all courses they have taken.
grouped = df.groupby('uni')